## Create renderable html text

* Get archives from common crawl and extract full html text
* Extract relevant html parts: css-links style tags and original table code
* Get Wayback Machine Internet Archive Link to access archived CSS files
* Create renderable html text

In [1]:
import gs as gs
import sqlite3
import pandas as pd
import imgkit
import os
import re
import json
import boto3 
import botocore
from bs4 import BeautifulSoup, Doctype
from warcio import ArchiveIterator
import urllib.request
import numpy as np
import pickle

In [2]:
os.chdir(r'C:\Users\babet\Documents\Studi\Master Thesis\Table_detection')

#load gold standard dataset

df = gs.load_gs(r"C:\Users\babet\Documents\Studi\Master Thesis\Table_detection\Data\mannheim.db")


In [ ]:
sdf=df
#sdf

### Access Common Crawl

In [5]:
#delete old bucket name [commoncrawl]
sdf.s3Link= sdf.s3Link.str.replace(r"common-crawl/", r"")

In [6]:
#import credentials for aws server
with open('credentials.json') as creds:    
 credentials = json.load(creds)

In [9]:
#use boto3
session = boto3.Session(
    aws_access_key_id=credentials['aws_access_key'],
    aws_secret_access_key=credentials['aws_secret_key'],
)
# define Amazon S3
s3 = session.resource('s3')

In [10]:
html_offset= []

for x in sdf.index:
        
    #assign variables
    key= sdf.s3Link.iloc[x]
    ra='bytes={}-{}'.format(sdf.recordOffset.iloc[x], str(int(sdf.recordEndOffset.iloc[x])+100000))
    url= sdf.url.iloc[x]

    #Get html common crawl using offsets 
    obj = s3.Object(bucket_name='commoncrawl', key=key)
    response = obj.get(Range=ra)
    data = response['Body']

    for record in ArchiveIterator(data):
          if record.rec_type == 'response':
                if record.rec_headers.get_header('WARC-Target-URI') == url:
                #f record.http_headers.get_header('Content-Type') == 'text/html':
                    #print(record.rec_headers.get_header('WARC-Target-URI')) 
                    html_offset.append(record.content_stream().read())
                    #print(html)
                    break


In [11]:
sdf['fullHtmlCode'] = html_offset

In [14]:
#External disk
os.chdir(r'E:\Babette\MasterThesis')
sdf.to_pickle('gs_log.pkl')  


The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.


### Extract relevant html parts

* Extract links to css files
* Complete relative links
* Extract style tags
* Extract full table code

In [3]:
html= []

for x in sdf.index:
    #html.append(BeautifulSoup(html_offset[x], 'html.parser'))
    html.append(BeautifulSoup(sdf.fullHtmlCode.iloc[x], 'html.parser'))
    #print(html.prettify())

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

In [4]:
links= []

for i in html:
    #find all css links
    for head in i.find_all("head"):
        link= head.find_all('link')
    longString=str()
    for x in link:
        longString = longString + " \n "+ str(x)
    links.append(longString)
    #links  


In [5]:
sdf['links'] = links

In [6]:
style= []

for i in html:
    #find all style tags
    for head in i.find_all("head"):
        sty= head.find_all('style')
    longString=str()
    for x in sty:
        longString = longString + " \n "+ str(x)
    style.append(longString)
    #style

In [7]:
#create new collumn
sdf['styleTag'] = style

In [10]:
#External disk
os.chdir(r'E:\Babette\MasterThesis')
sdf.to_pickle('gs_log.pkl')  


In [ ]:
fullTable= []
error= [1705, 4054,4362] #find out why!
for i in html:
    x= html.index(i)
    if x in error:
        fullTable.append("NAN")
    else:        
        #print(x)
        tables= list()
        for tag in i.find_all(re.compile("table")):
            tables.append(tag)
        fullTable.append(str(tables[int(sdf.tableNum.iloc[x])]))
    #fullTable

In [13]:
#create new collumn
sdf['fullTable'] = fullTable

### Create full renderable html Code

In [27]:
renderCode =[]

for x in sdf.index:
    links= sdf.links.iloc[x]
    styleTag= sdf.styleTag.iloc[x]
    if sdf.fullTable.iloc[x] =="NAN":
        fullTable= "<table>" +sdf.htmlCode.iloc[x]+ "</table"
    else:
        fullTable= sdf.fullTable.iloc[x]
    
    #Create Table html code including links & style info
    htmlAdd= r'<html> <head>'+ links + styleTag + ' </head> <body>  '+ fullTable + r' </body> </html>'
    
    renderCode.append(htmlAdd)

In [28]:
#create new collumn
sdf['renderCode'] = renderCode

In [29]:
#create complete links if necessary
links_comp= []

for x in sdf.index:
    #url= sdf.url.iloc[x]
    url= re.sub(r'/$', r'', sdf.url.iloc[x])
    #print(url)
    
    #case 1 current directory
    links1=re.sub('href="(?!(/|\.\.|http))', r'href="'+ url+ r'/', sdf.renderCode.iloc[x])

    #case 2: current root
    root_url= re.match('https?://(?:.*\.)*(.+\..+?)/', sdf.url.iloc[x]).group(0)
    #short_url= re.sub(r"/(?:.(?!/))+$", r"/", url)
    #print(short_url)
    links2=re.sub(r'href="/(?!/)', r'href="'+ root_url, links1)

    #case 3: two above
    shorter_url= re.sub(r"/[^/]*/(?:.(?!/))+$", r"/", url)
    links3=re.sub(r'href="../', r'href="'+ shorter_url, links2)
    #print(shorter_url)
    
    #case4: autonomous link //
    links4=re.sub(r'href="//', r'href="http://', links3)

    
    #same for style links
    
    #case 1 current directory
    links5=re.sub('url\((\\\')(?!(/|\.\.|\"http|http|https|\"https))', r'url(\''+ url+ r'/', links4)
    links6=re.sub('url\((?!(/|\.\.|\"http|http|https|\"https|\\\'))', r'url('+ url+ r'/', links5)

    #case 2: current root
    #short_url= re.sub(r"/(?:.(?!/))+$", r"/", url)
    #print(short_url)
    links7=re.sub('url\(\\*\'*/(?!/)', r'url(\''+ root_url, links6)
    links8=re.sub('url\(/(?!/)', r'url('+ root_url, links7)

    #case 3: two above
    shorter_url= re.sub(r"/[^/]*/(?:.(?!/))+$", r"/", url)
    links9=re.sub('url\(\\*\'*\.\./', r'url(\''+ shorter_url, links8)
    links10=re.sub('url\(\.\./', r'url('+ shorter_url, links9)
    #print(shorter_url)
    
    #case 4 autonomous link
    links11=re.sub('url\(//', r'url(http://', links10)
    
    #caseX scr
    links12=re.sub('src=\"(?!(/|\.\.|\"http|http|https|\"https|\\\'))', r'src="'+ url+ r'/', links11)
    links13=re.sub('src=\"/(?!/)', r'src="'+ root_url, links12)
    links14=re.sub('src=\"\.\./', r'src="'+ shorter_url, links13)
    links15=re.sub('src=\"//', r'src="http://', links14)
    
    
    #links6=links3
    links_comp.append(links15)

In [30]:
sdf['renderCodeLink'] = links_comp

### Get wayback machine path to access css files

In [ ]:
archivePath = []
error=[2990] #find out why!

for x in sdf.index:    
    print(x)
    req_string= "http://web.archive.org/wayback/available?url="+ str(sdf.url.iloc[x]) + "&timestamp=20140101"
    contents = urllib.request.urlopen(req_string).read()
    #print(contents)
    dic = json.loads(contents)
    if x in error:
        archivePath.append("NAN")
    elif 'closest' in dic['archived_snapshots']:
        archive_url =dic['archived_snapshots']['closest']['url']
        #archive = re.sub(sdf.url.iloc[x], r'', archive_url)
        archive = re.match(r'http://web.archive.org/web/\d*/', archive_url )
        #print(archive.group(0))
        archivePath.append(str(archive.group(0)))

        
    else:
        archivePath.append("NAN")

In [26]:
#create new collumn
sdf['archivePath'] = archivePath

In [31]:
#add paths to archived css documents to renderable Code
renderCodeImage= []

for x in sdf.index:
    archivePath= sdf.archivePath.iloc[x]
    html = sdf.renderCodeLink.iloc[x]
    
    if archivePath!= 'NAN':
        htmlA=re.sub(r'"http:', r'"'+ archivePath+ 'http:', html)
        htmlArchive=re.sub('\(http:', '\('+ archivePath+ 'http:', htmlA)
    else:
        htmlArchive=html
    renderCodeImage.append(htmlArchive)
    

In [32]:
sdf['renderCodeImage']= renderCodeImage

### Save as pickle

In [34]:
sdf.to_pickle('gs_renderable.pkl')  

#ssdf = pd.read_pickle('gs_renderable_subset.pkl')

In [35]:
sdf.archivePath.describe()

count     5777
unique    2935
top        NAN
freq      2820
Name: archivePath, dtype: object